# Text Main For Language Model

> This module contains the main Python class for Language Model data control: `TextDataLMController`

- skip_showdoc: true
- skip_exec: true

In [ ]:
#| default_exp text_main_lm

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from __future__ import annotations
from datasets import DatasetDict,Dataset,IterableDataset
from pathlib import Path
from that_nlp_library.utils import *
from that_nlp_library.text_main import *
from functools import partial
import warnings
from transformers import DataCollatorForLanguageModeling

In [ ]:
import pandas as pd
import numpy as np
from that_nlp_library.text_transformation import *
from that_nlp_library.text_augmentation import *
from importlib.machinery import SourceFileLoader
from datasets import load_dataset
import os

## Class TextDataLMController

In [ ]:
#| export
class TextDataLMController(TextDataController):
    def __init__(self,
                 inp, # HuggingFainpce Dataset or DatasetDict
                 main_text:str, # Name of the main text column
                 filter_dict={}, # A dictionary: {feature: filtering_function_for_that_feature}
                 metadatas=[], # Names of the metadata columns
                 process_metas=True, # Whether to do simple text processing on the chosen metadatas
                 content_transformations=[], # A list of text transformations
                 val_ratio:int|float|None=0.2, # Ratio of data for validation set
                 stratify_cols=[], # Column(s) needed to do stratified shuffle split
                 seed=None, # Random seed
                 batch_size=1024, # CPU batch size
                 num_proc=4, # Number of process for multiprocessing
                 cols_to_keep=None, # Columns to keep after all processings
                 verbose=True, # Whether to prdint processing information
                ):
        super().__init__(inp=inp,
                         main_text=main_text,
                         filter_dict=filter_dict,
                         metadatas=metadatas,
                         process_metas=process_metas,
                         content_transformations=content_transformations,
                         val_ratio=val_ratio,
                         stratify_cols=stratify_cols,
                         seed=seed,
                         batch_size=batch_size,
                         num_proc=num_proc,
                         cols_to_keep=cols_to_keep,
                         verbose=verbose
                        )
            
    
    def _do_label_transformation(self):
        raise NotImplementedError("There's no classification/regression label in text processing for Language Model")
        
    def _encode_labels(self):
        raise NotImplementedError("There's no classification/regression label in text processing for Language Model")

    
    def _upsampling(self):
        raise NotImplementedError("There's no upsampling in text processing for Language Model")
      
    def _do_augmentation(self):
        raise NotImplementedError("There's no text augmentation in text processing for Language Model")
     
    def save_as_pickles(self,
                        fname, # Name of the pickle file
                        parent='pickle_files', # Parent folder
                       ):
        
        save_to_pickle(self,fname,parent=parent) 
        
    def _do_train_shuffling(self):
        print_msg('Shuffling and flattening train set',20,verbose=self.verbose)
        self.main_ddict['train'] = self.main_ddict['train'].shuffle(seed=self.seed).flatten_indices(num_proc = self.num_proc)
        self.verboseprint('Done')

    def _group_texts_with_stride(self,examples):
        max_length = self.max_length
        if max_length is None: 
            max_length = self.tokenizer.model_max_length
        stride = self.stride
        if stride is None: stride=max_length
        else: stride = max_length-stride
        if stride==0: raise ValueError(f'Stride cannot be equal to max length of {max_length}')
            
        # Concatenate all texts.
        concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
        total_length = len(concatenated_examples[list(examples.keys())[0]])
        result_all={}
        for k,t in concatenated_examples.items():
            result=[]
            i=0
            while i+max_length<=total_length:
                result.append(t[i:i+max_length])
                i+=stride
            result_all[k]=result

        return result_all  
    

    def do_all_preprocessing(self,
                             shuffle_trn=True # To shuffle the train set before tokenization
                            ):
        if self._processed_call:
            warnings.warn('Your dataset has already been processed. Returning the previous processed DatasetDict...')
            return self.main_ddict
            
        print_msg('Start Main Text Processing',20,verbose=self.verbose)
        
        # Filtering
        self.dset,self.ddict_rest = self._do_filtering(self.dset,self.ddict_rest)
        
        # Process metadatas
        self.dset,self.ddict_rest = self._process_metadatas(self.dset,self.ddict_rest)
        
        
        # Content transformation
        self.dset,self.ddict_rest = self._do_transformation(self.dset,self.ddict_rest)
         
        # Train Test Split.
        ### self.main_ddict is created here
        self._train_test_split()
        
        # Dropping unused columns
        self._simplify_ddict()
        
        # Check validation leaking
        self._check_validation_leaking()
        
        # Shuffle train
        if shuffle_trn:
            self._do_train_shuffling()
        
        self._processed_call=True
        
        return self.main_ddict
    
        
    def do_tokenization(self,
                        tokenizer, # Tokenizer (preferably from HuggingFace)
                        max_length=None, # pad to model's allowed max length (default is max_sequence_length). Use -1 for no padding at all
                        line_by_line=True, # To whether tokenize each sentence separately, or concatenate them
                        stride=None, # option to do striding when line_by_line is False
                        trn_size=None, # The number of training data to be tokenized
                        tok_num_proc=None, # Number of processes for tokenization
                       ):
        # References
#         https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
#         https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_clm.py
        
        print_msg('Tokenization',20,verbose=self.verbose)
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.line_by_line = line_by_line
        self.stride = stride
        self.tok_num_proc = tok_num_proc if tok_num_proc else self.num_proc
        
        tok_func = partial(tokenize_function,tok=self.tokenizer,
                           max_length=max_length if line_by_line else -1,
                           return_special_tokens_mask=True)
        
        _func = partial(lambda_map_batch,
                        feature=self.main_text,
                        func=tok_func,
                        output_feature=None,
                        is_batched=self.is_batched)
        
        
        if trn_size is not None:
            if isinstance(trn_size,float):
                num_shard = int(1/trn_size)
            else: # int
                trn_len=len(self.main_ddict['train'])
                num_shard = trn_len//trn_size
            self.main_ddict['train'] = self.main_ddict['train'].shard(num_shard,0)
        
        for k in self.main_ddict.keys():
            self.main_ddict[k] = hf_map_dset(self.main_ddict[k],_func,self.is_batched,self.batch_size,self.tok_num_proc)
            self.main_ddict[k] = self.main_ddict[k].remove_columns(self.cols_to_keep)
        
        if not line_by_line: # token concatenation
            for k in self.main_ddict.keys():
                self.main_ddict[k] = hf_map_dset(self.main_ddict[k],
                                                 self._group_texts_with_stride,
                                                 is_batched=True,
                                                 batch_size=self.batch_size if self.batch_size>1 else 1024,
                                                 num_proc=self.tok_num_proc)
                
        
        self.verboseprint('Done')
        return self.main_ddict
        
    def process_and_tokenize(self,
                             tokenizer, # Tokenizer (preferably from HuggingFace)
                             max_length=None, # pad to model's allowed max length (default is max_sequence_length)
                             line_by_line=True, # To whether tokenize each sentence separately, or concatenate them and then tokenize
                             stride=None, # option to do striding when line_by_line is False
                             trn_size=None, # The number of training data to be tokenized
                             tok_num_proc=None, # Number of processes for tokenization
                             shuffle_trn=True, # To shuffle the train set before tokenization
                            ):
        """
        This will perform `do_all_processing` then `do_tokenization`
        """
        _ = self.do_all_preprocessing(shuffle_trn)
        _ = self.do_tokenization(tokenizer,max_length,line_by_line,stride,trn_size,tok_num_proc)
        
    
    def set_data_collator(self,
                          is_mlm=True, # Is this masked language model (True) or causal language model (False)
                          mlm_prob=0.15, # Mask probability for masked language model
                         ):
        if not hasattr(self,'max_length'):
            raise ValueError("Please call `process_and_tokenize' or `do_tokenization` to tokenize your dataset")
        
        self.is_mlm = is_mlm
        pad_to_multiple_of_8 = (self.max_length<0) # get data collator to pad
        self.data_collator = DataCollatorForLanguageModeling(tokenizer=self.tokenizer,
                                                             mlm=is_mlm,
                                                             mlm_probability=mlm_prob,
                                                             pad_to_multiple_of=8 if pad_to_multiple_of_8 else None
                                                            )
                                               
    
    def prepare_test_dataset_from_raws(self,
                                       content, # Either a single sentence, list of sentence or a dictionary with keys are metadata columns and values are list
                                      ):
        if len(self.metadatas) and not isinstance(content,dict):
            raise ValueError(f'There is/are metadatas in the preprocessing step. Please include a dictionary including these keys for metadatas: {self.metadatas}, and texture content: {self.main_text}')
            
        _dic = {self.main_text:[content]} if isinstance(content,str) else content
        for k in _dic.keys():
            _dic[k] = val2iterable(_dic[k])
        
        test_dict = Dataset.from_dict(_dic)
        
        # set num_proc to 1 for small data processing
        _tmp1 = self.num_proc
        _tmp2 = self.tok_num_proc
        self.num_proc=1
        self.tok_num_proc=1
        results = self.prepare_test_dataset(test_dict)
        self.num_proc = _tmp1
        self.tok_num_proc=_tmp2
        return results
        
    def prepare_test_dataset(self,
                             test_dset, # The HuggingFace Dataset as Test set
                            ):
        test_cols = set(get_dset_col_names(test_dset))
        missing_cols = set(self.cols_to_keep) - test_cols
        if len(missing_cols):
            raise ValueError(f'Test set does not have these columns required for preprocessings: {missing_cols}')
            
        print_msg('Start Test Set Transformation',20,verbose=self.verbose)
        
        # Process metadatas
        test_dset = self._process_metadatas(test_dset)
        
        # Content transformation
        test_dset = self._do_transformation(test_dset)
        
        # Drop every columns except for main_text
        cols_to_remove = {c for c in test_cols if c!=self.main_text}
        test_dset=test_dset.remove_columns(list(cols_to_remove))
        
        self.verboseprint('Done')
        return test_dset

In [ ]:
show_doc(TextDataLMController)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main_lm.py#L17){target="_blank" style="float:right; font-size:smaller"}

### TextDataLMController

>      TextDataLMController (inp, main_text:str, filter_dict={}, metadatas=[],
>                            process_metas=True, content_transformations=[],
>                            val_ratio:int|float|None=0.2, stratify_cols=[],
>                            seed=None, batch_size=1024, num_proc=4,
>                            cols_to_keep=None, verbose=True)

Initialize self.  See help(type(self)) for accurate signature.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| inp |  |  | HuggingFainpce Dataset or DatasetDict |
| main_text | str |  | Name of the main text column |
| filter_dict | dict | {} | A dictionary: {feature: filtering_function_for_that_feature} |
| metadatas | list | [] | Names of the metadata columns |
| process_metas | bool | True | Whether to do simple text processing on the chosen metadatas |
| content_transformations | list | [] | A list of text transformations |
| val_ratio | int \| float \| None | 0.2 | Ratio of data for validation set |
| stratify_cols | list | [] | Column(s) needed to do stratified shuffle split |
| seed | NoneType | None | Random seed |
| batch_size | int | 1024 | CPU batch size |
| num_proc | int | 4 | Number of process for multiprocessing |
| cols_to_keep | NoneType | None | Columns to keep after all processings |
| verbose | bool | True | Whether to prdint processing information |

## Load data + Basic use case

In [ ]:
show_doc(TextDataLMController.from_csv)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.from_csv

>      TextDataController.from_csv (file_path, **kwargs)

In [ ]:
show_doc(TextDataLMController.from_df)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.from_df

>      TextDataController.from_df (df, validate=True, **kwargs)

You can create a `TextDataLMController` from a csv, pandas DataFrame, or directly from a HuggingFace dataset object. Currently, `TextDataLMController` is designed for processing text in order to train a language model


Dataset source: https://www.kaggle.com/datasets/kavita5/review_ecommerce

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')

In [ ]:
df.shape

(23486, 10)

In [ ]:
df.sample(5) 

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
11667,1078,37,NaN,I also agree with the other reviewers that it'...,4,1,6,General,Dresses,Dresses
1682,860,56,Great summer t,"Great t. great lightweight, almost linen like ...",4,1,1,General Petite,Tops,Knits
4414,1081,20,"Great dress, comfortable and cute",This dress is so comfortable and versatile. yo...,4,1,0,General Petite,Dresses,Dresses
10919,1083,53,Beautiful,This is my third dress by this designer. i lov...,5,1,1,General,Dresses,Dresses
5645,1099,23,Flattering,"I am short (5'3'"") and have a short torso so t...",5,1,0,General Petite,Dresses,Dresses


You can create a `TextDataLMController` from a dataframe. This also provides a quick input validation check (NaN check and Duplication check)

In [ ]:
tdc = TextDataLMController.from_df(df,main_text='Review Text')

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows


You can also create a `TextDataLMController` directly from the csv file. The good thing about using HuggingFace Dataset as the main backend  is that you can utilize lots of its useful functionality, such as caching

In [ ]:
tdc = TextDataLMController.from_csv('sample_data/Womens_Clothing_Reviews.csv',main_text='Review Text')

You can also create a `TextDataLMController` from a HuggingFace Dataset

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
dset

Dataset({
    features: ['Clothing ID', 'Age', 'Title', 'Review Text', 'Rating', 'Recommended IND', 'Positive Feedback Count', 'Division Name', 'Department Name', 'Class Name'],
    num_rows: 23486
})

In [ ]:
tdc = TextDataLMController(dset,main_text='Review Text')

In the "Input Validation Precheck" above, we notice that our dataset has missing values in the text field and the label field. For now, let's load the data as a Pandas' DataFrame, perform some cleaning, and create our `TextDataLMController`

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')

In [ ]:
df = df[(~df['Review Text'].isna()) & (~df['Department Name'].isna())].reset_index(drop=True)

In [ ]:
tdc = TextDataLMController.from_df(df,main_text='Review Text')

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title    2966
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 1 rows


At this point you can start perform 2 important steps on your data

1. Text preprocessings + Train/Validation Split
2. Tokenization

In [ ]:
ddict = tdc.do_all_preprocessing(shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.02% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18098 [00:00<?, ? examples/s]

Done


In [ ]:
ddict

DatasetDict({
    train: Dataset({
        features: ['Review Text'],
        num_rows: 18098
    })
    validation: Dataset({
        features: ['Review Text'],
        num_rows: 4526
    })
})

Our DatasetDict now has two split: train and validation. Note that train split is now IterableDataset, for processing efficiency

In [ ]:
ddict['train'][:3]

{'Review Text': ["I like the idea of the pullover. the quality of is nice; the material is very soft and feels pleasant to the touch.\r\n\r\ni expected this sweater to be more flattering though. it is true to size, but the length is strange. it seems a bit too long, yet is too short to look like a longer tunic. or maybe the waistline is a little low... whatever it is, it makes me look like i have short legs.\r\n\r\nalso, the collar won't stay down, the way it is shown in the picture. but it may work better for someone wi",
  "The color and the fabric in these shorts is quite nice as is the length. i didn't order the pilcro bermudas because another reviewer said the legs were quite wide. unfortunately, these shorts also have wide legs. i'm going to have them tailored so they hug the legs as in the picture rather than billowing out. i do not have skinny thighs(don't i wish!) and i was surprised that the fit was not as pictured. if you're ok with wide legs or can get them at a price at wh

In [ ]:
ddict['validation'][:3]

{'Review Text': ["Ok, this shirt is like my dream.  it's large, comfy but sexy with the back dip.  as a larger chested lady (32g), i really appreciate having a tee that i can easily throw on and look put together.  clearly, this is a larger, styled swing tee but it's one of my favorites.  paired with a cute pair of boots, and easily worn with a jacket.  it can tend to fall off the shoulders so i'd size down.  i'll probably pick up the grey too.",
  'Truly the only word that comes to mind when i think of this dress is dazzling! the pictures online just barely do it justice...if you have an retailer nearby, check to see if its available in store and go check it out in person. it does not disappoint! every year around this time, our beloved retailer seems to come out with this style of dress. in the years past they have all been different and beautiful, but i think they have truly outdone themselves this year. this dress is a deep emerald green...',
  "My main complaint, is that the butto

## Filtering

This preprocessing step allow you to filter out certain values of a certain column in your dataset. Let's say I want to filter out any None value in the column 'Review Text'

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')
df[(~df['Review Text'].isna())].isna().sum()

Clothing ID                   0
Age                           0
Title                      2966
Review Text                   0
Rating                        0
Recommended IND               0
Positive Feedback Count       0
Division Name                13
Department Name              13
Class Name                   13
dtype: int64

We will provide a dictionary containing the name of the column and the filtering function to apply on that column. Note that **the filtering function will receive an item from the column, and the function should return a boolean**

In [ ]:
tdc = TextDataLMController.from_df(df,
                                 main_text='Review Text',
                                 filter_dict={'Review Text': lambda x: x is not None},
                                 seed=42
                                )

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows


In [ ]:
ddict = tdc.do_all_preprocessing(shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 1, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18112 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18111 [00:00<?, ? examples/s]

Done


In [ ]:
ddict

DatasetDict({
    train: Dataset({
        features: ['Review Text'],
        num_rows: 18111
    })
    validation: Dataset({
        features: ['Review Text'],
        num_rows: 4529
    })
})

Let's check if we have filtered out all NaN/None value

In [ ]:
for i in ddict['train']['Review Text']:
    assert i is not None
for i in ddict['validation']['Review Text']:
    assert i is not None

We can even add multiple filtering functions. Remember from our precheck, there are also None values in 'Department Name'. While we are at it, let's filter out any rating that is less than 3 (just to showcase what our filtering can do)

In [ ]:
df.Rating.value_counts()

Rating
5    13131
4     5077
3     2871
2     1565
1      842
Name: count, dtype: int64

Note that `TextDataLMController` will only keep the text and the metadatas columns; any other column will be dropped. To double-check our result, we need to define the `cols_to_keep` argument

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')
tdc = TextDataLMController.from_df(df,
                                   main_text='Review Text',
                                   filter_dict={'Review Text': lambda x: x is not None,
                                                'Department Name': lambda x: x is not None,
                                                'Rating': lambda x: x>=3
                                               },
                                   cols_to_keep=['Review Text','Rating','Department Name'],
                                   seed=42
                                  )

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows


In [ ]:
ddict = tdc.do_all_preprocessing(shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

----- Do <lambda> on Rating -----


...

Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 1, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/16206 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/16205 [00:00<?, ? examples/s]

Done


In [ ]:
for i in ddict['train']['Department Name']:
    assert i is not None
for i in ddict['validation']['Department Name']:
    assert i is not None

for i in ddict['train']['Rating']:
    assert i is not None
for i in ddict['validation']['Rating']:
    assert i >= 3

## Metadatas concatenation

If we think metadatas can be helpful, we can concatenate them into the front of your text, so that our text classification model is aware of it.

In this example, Let's add 'Title' as our metadata

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')
tdc = TextDataLMController.from_df(df,
                                   main_text='Review Text',
                                   filter_dict={'Review Text': lambda x: x is not None},
                                   metadatas='Title',
                                   process_metas=True, # to preprocess the metadata (currently it's just empty space stripping and lowercasing),
                                   seed=42
                                  )

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows


In [ ]:
ddict = tdc.do_all_preprocessing(shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

Done
----- Metadata Simple Processing & Concatenating to Main Content -----


Map (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 0, which is 0.00% of training set
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18112 [00:00<?, ? examples/s]

Done


In [ ]:
ddict['train'][:3]

{'Title': ['not flattering on me', '', ''],
 'Review Text': ['not flattering on me . I ordered this online and was disappointed with the fit when it arrived. i ordered the xs and it was still oversize to the point of being unflattering. i am tall 5\'9" about 130 pounds and have a fairly thin torso and look best in cloths that have some shape. if you like a loose fit this might be for you. the material is thicker and warm and comfortable. i would suggest ordering down a size.',
  " . So unflattering! really disappointed. made me look 6 month pregnant and i'm a petite size 2.",
  ' . This t-shirt does a great job of elevating the basic t-shirt in to one with a touch of flair. i typically wear a medium but luckily read earlier reviews and went with the small.']}

In [ ]:
ddict['validation'][:3]

{'Title': ['', '', ''],
 'Review Text': [" . This picture doesn't do the skirt justice. i paired it with a creme colored cashmere cowlneck sweater and a silver jeweled belt. it is really pretty and flattering on.",
  ' . Easy to wear! cute, comfy...will be a go to for summer.',
  ' . Nice sweater, just did not look good on me. sorry, going back.']}

## Content Transformation

This processing allows you to **alter the text content in your dataset**. You need to define a function that accepts a single string and returns a new, processed string. Note that this transformation will be applied to ALL of your dataset (both train and validation)

Let's say we want to normalize our text, because the text might contain some extra spaces between words, or not follow the "single space after a period" rule

In [ ]:
_tmp = "This is a      sentence,which doesn't follow any rule!No single space is provided after period or punctuation marks.    Maybe there are too many spaces!?!   "

In [ ]:
from underthesea import text_normalize

In [ ]:
text_normalize(_tmp)

"This is a sentence , which doesn't follow any rule ! No single space is provided after period or punctuation marks . Maybe there are too many spaces ! ? !"

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataLMController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         content_transformations=text_normalize,
                         seed=42
                        )

In [ ]:
ddict = tdc.do_all_preprocessing(shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
Done
-------------------- Text Transformation --------------------
----- text_normalize -----
Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 1, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18112 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18111 [00:00<?, ? examples/s]

Done


In [ ]:
ddict['train']['Review Text'][0]

'I ordered this online and was disappointed with the fit when it arrived . i ordered the xs and it was still oversize to the point of being unflattering . i am tall 5 \' 9 " about 130 pounds and have a fairly thin torso and look best in cloths that have some shape . if you like a loose fit this might be for you . the material is thicker and warm and comfortable . i would suggest ordering down a size .'

In [ ]:
ddict['validation']['Review Text'][0]

"This picture doesn't do the skirt justice . i paired it with a creme colored cashmere cowlneck sweater and a silver jeweled belt . it is really pretty and flattering on ."

You can chain multiple functions. Let's say after text normalizing, I want to lowercase the text

In [ ]:
str.lower('tHis IS NoT lowerCASE')

'this is not lowercase'

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataLMController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         content_transformations=[text_normalize,str.lower],
                         seed=42
                        )

In [ ]:
ddict = tdc.do_all_preprocessing(shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----
Done
-------------------- Text Transformation --------------------
----- text_normalize -----
----- lower -----
Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 1, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/18112 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18111 [00:00<?, ? examples/s]

Done


In [ ]:
ddict['train']['Review Text'][0]

'i ordered this online and was disappointed with the fit when it arrived . i ordered the xs and it was still oversize to the point of being unflattering . i am tall 5 \' 9 " about 130 pounds and have a fairly thin torso and look best in cloths that have some shape . if you like a loose fit this might be for you . the material is thicker and warm and comfortable . i would suggest ordering down a size .'

In [ ]:
ddict['validation']['Review Text'][0]

"this picture doesn't do the skirt justice . i paired it with a creme colored cashmere cowlneck sweater and a silver jeweled belt . it is really pretty and flattering on ."

## Train/Validation Split

There are several ways to perform a train/validation split with `TextDataLMController`

The first way is when you already have a validation split in your HuggingFace's Dataset. Let's use the Dataset built-in function `train_test_split` to simulate this

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
ddict_with_val = dset.train_test_split(test_size=0.1)
# This will create a 'test' split instead of 'validation', so we will process a bit to have a validation split
ddict_with_val['validation']=ddict_with_val['test']
del ddict_with_val['test']

In [ ]:
ddict_with_val

DatasetDict({
    train: Dataset({
        features: ['Clothing ID', 'Age', 'Title', 'Review Text', 'Rating', 'Recommended IND', 'Positive Feedback Count', 'Division Name', 'Department Name', 'Class Name'],
        num_rows: 21137
    })
    validation: Dataset({
        features: ['Clothing ID', 'Age', 'Title', 'Review Text', 'Rating', 'Recommended IND', 'Positive Feedback Count', 'Division Name', 'Department Name', 'Class Name'],
        num_rows: 2349
    })
})

In [ ]:
tdc = TextDataLMController(ddict_with_val,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         seed=42
                        )
ddict = tdc.do_all_preprocessing(shuffle_trn=True)

-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/21137 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/2349 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Validation split already exists
Done
-------------------- Dropping unused features --------------------
Done
- Number of rows leaked: 3, which is 0.01% of training set
Filtering leaked data out of training set...


Filter (num_proc=4):   0%|          | 0/20385 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/20382 [00:00<?, ? examples/s]

Done


In [ ]:
ddict

DatasetDict({
    train: Dataset({
        features: ['Review Text'],
        num_rows: 20382
    })
    validation: Dataset({
        features: ['Review Text'],
        num_rows: 2256
    })
})

A second way is to split randomly based on a ratio (a float between 0 and 1), or based on the number of data in your validation set

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataLMController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         val_ratio=0.15,
                         seed=42,
                         verbose=False
                        )
ddict = tdc.do_all_preprocessing(shuffle_trn=True)
ddict

DatasetDict({
    train: Dataset({
        features: ['Review Text'],
        num_rows: 19243
    })
    validation: Dataset({
        features: ['Review Text'],
        num_rows: 3397
    })
})

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataLMController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         val_ratio=5000,
                         seed=42,
                         verbose=False
                        )
ddict = tdc.do_all_preprocessing(shuffle_trn=True)
ddict

DatasetDict({
    train: Dataset({
        features: ['Review Text'],
        num_rows: 17640
    })
    validation: Dataset({
        features: ['Review Text'],
        num_rows: 5000
    })
})

A third way is to do a random stratified split (inspired by [sklearn's](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)). Let's do a stratified split based on our label 'Department Name'

In [ ]:
df = pd.read_csv('sample_data/Womens_Clothing_Reviews.csv',encoding='utf-8-sig')

In [ ]:
df['Department Name'].value_counts(normalize=True)

Department Name
Tops        0.445978
Dresses     0.269214
Bottoms     0.161852
Intimate    0.073918
Jackets     0.043967
Trend       0.005070
Name: proportion, dtype: float64

In [ ]:
tdc = TextDataLMController.from_df(df,
                                 main_text='Review Text',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 val_ratio=0.2,
                                 stratify_cols='Department Name',
                                 cols_to_keep=['Review Text','Department Name'],
                                 seed=42
                                )
ddict = tdc.do_all_preprocessing(shuffle_trn=True)
ddict

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows
-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

----- Do <lambda> on Department Name -----


Filter (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
Validation split based on val_ratio, with stratifying


...

Filter (num_proc=4):   0%|          | 0/18102 [00:00<?, ? examples/s]

Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/18100 [00:00<?, ? examples/s]

Done


DatasetDict({
    train: Dataset({
        features: ['Review Text', 'Department Name'],
        num_rows: 18100
    })
    validation: Dataset({
        features: ['Review Text', 'Department Name'],
        num_rows: 4526
    })
})

In [ ]:
pd.Series(ddict['train']['Department Name']).value_counts(normalize=True)

Tops        0.444033
Dresses     0.271602
Bottoms     0.161878
Intimate    0.072983
Jackets     0.044309
Trend       0.005193
Name: proportion, dtype: float64

In [ ]:
pd.Series(ddict['validation']['Department Name']).value_counts(normalize=True)

Tops        0.444101
Dresses     0.271542
Bottoms     0.161732
Intimate    0.073133
Jackets     0.044189
Trend       0.005303
Name: proportion, dtype: float64

You can also use multiple columns for your stratification

In [ ]:
tdc = TextDataLMController.from_df(df,
                                 main_text='Review Text',
                                 filter_dict={'Review Text': lambda x: x is not None,
                                              'Department Name': lambda x: x is not None,
                                             },
                                 val_ratio=0.2,
                                 stratify_cols=['Department Name','Rating'],
                                 cols_to_keep=['Review Text','Department Name','Rating'],
                                 seed=42,
                                 verbose=False
                                )
ddict = tdc.do_all_preprocessing(shuffle_trn=True)
ddict

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows


DatasetDict({
    train: Dataset({
        features: ['Review Text', 'Rating', 'Department Name'],
        num_rows: 18100
    })
    validation: Dataset({
        features: ['Review Text', 'Rating', 'Department Name'],
        num_rows: 4526
    })
})

And finally, you can omit any validation split if you specify `val_ratio` as ```None```

In [ ]:
tdc = TextDataLMController.from_df(df,
                                 main_text='Review Text',
                                 filter_dict={'Review Text': lambda x: x is not None},
                                 val_ratio=None,
                                 seed=42
                                )
ddict = tdc.do_all_preprocessing(shuffle_trn=True)
ddict

- Input Validation Precheck -
Data contains missing values!
-----> List of columns and the number of missing values for each
Title              3810
Review Text         845
Division Name        14
Department Name      14
Class Name           14
dtype: int64
Data contains duplicated values!
-----> Number of duplications: 21 rows
-------------------- Start Main Text Processing --------------------
-------------------- Data Filtering --------------------
----- Do <lambda> on Review Text -----


Filter (num_proc=4):   0%|          | 0/23486 [00:00<?, ? examples/s]

Done
-------------------- Train Test Split --------------------
No validation split defined
Done
-------------------- Dropping unused features --------------------
Done
-------------------- Shuffling and flattening train set --------------------


Flattening the indices (num_proc=4):   0%|          | 0/22641 [00:00<?, ? examples/s]

Done


DatasetDict({
    train: Dataset({
        features: ['Review Text'],
        num_rows: 22641
    })
})

## Tokenization

Define our tokenization

In [ ]:
from transformers import RobertaTokenizer
from underthesea import text_normalize

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [ ]:
show_doc(TextDataLMController.process_and_tokenize)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main_lm.py#L190){target="_blank" style="float:right; font-size:smaller"}

### TextDataLMController.process_and_tokenize

>      TextDataLMController.process_and_tokenize (tokenizer, max_length=None,
>                                                 line_by_line=True,
>                                                 stride=None, trn_size=None,
>                                                 tok_num_proc=None,
>                                                 shuffle_trn=True)

This will perform `do_all_processing` then `do_tokenization`

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| tokenizer |  |  | Tokenizer (preferably from HuggingFace) |
| max_length | NoneType | None | pad to model's allowed max length (default is max_sequence_length) |
| line_by_line | bool | True | To whether tokenize each sentence separately, or concatenate them and then tokenize |
| stride | NoneType | None | option to do striding when line_by_line is False |
| trn_size | NoneType | None | The number of training data to be tokenized |
| tok_num_proc | NoneType | None | Number of processes for tokenization |
| shuffle_trn | bool | True | To shuffle the train set before tokenization |

### Option 1: Tokenize our corpus line-by-line

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataLMController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         content_transformations=[text_normalize,str.lower],
                         seed=42,
                         verbose=False
                        )

With no padding

In [ ]:
tdc.process_and_tokenize(tokenizer,line_by_line=True,max_length=-1)

In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'special_tokens_mask', 'attention_mask'],
        num_rows: 18111
    })
    validation: Dataset({
        features: ['input_ids', 'special_tokens_mask', 'attention_mask'],
        num_rows: 4529
    })
})

In [ ]:
print(tokenizer.decode(tdc.main_ddict['train']['input_ids'][0]))
print(tokenizer.decode(tdc.main_ddict['validation']['input_ids'][0]))

<s>i ordered this online and was disappointed with the fit when it arrived. i ordered the xs and it was still oversize to the point of being unflattering. i am tall 5'9 " about 130 pounds and have a fairly thin torso and look best in cloths that have some shape. if you like a loose fit this might be for you. the material is thicker and warm and comfortable. i would suggest ordering down a size.</s>
<s>this picture doesn't do the skirt justice. i paired it with a creme colored cashmere cowlneck sweater and a silver jeweled belt. it is really pretty and flattering on.</s>


With padding (set `max_length` to `None` if you want to pad to model's maximum sequence length)

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataLMController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         content_transformations=[text_normalize,str.lower],
                         seed=42,
                         verbose=False
                        )

In [ ]:
tdc.process_and_tokenize(tokenizer,line_by_line=True,max_length=100)

In [ ]:
print(tokenizer.decode(tdc.main_ddict['train']['input_ids'][0]))
print(tokenizer.decode(tdc.main_ddict['validation']['input_ids'][0]))

<s>i ordered this online and was disappointed with the fit when it arrived. i ordered the xs and it was still oversize to the point of being unflattering. i am tall 5'9 " about 130 pounds and have a fairly thin torso and look best in cloths that have some shape. if you like a loose fit this might be for you. the material is thicker and warm and comfortable. i would suggest ordering down a size.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad>
<s>this picture doesn't do the skirt justice. i paired it with a creme colored cashmere cowlneck sweater and a silver jeweled belt. it is really pretty and flattering on.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


### Option 2: Tokenize every text, then concatenate them together before splitting them in smaller parts.


In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataLMController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         content_transformations=[text_normalize,str.lower],
                         seed=42,
                         verbose=False,
                        )

In [ ]:
tdc.process_and_tokenize(tokenizer,line_by_line=False,max_length=100)

In [ ]:
tdc.main_ddict

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'special_tokens_mask', 'attention_mask'],
        num_rows: 13573
    })
    validation: Dataset({
        features: ['input_ids', 'special_tokens_mask', 'attention_mask'],
        num_rows: 3446
    })
})

In [ ]:
for i in tdc.main_ddict['train']['input_ids'][:3]:
    print(tokenizer.decode(i))
    print('-'*100)

<s>i ordered this online and was disappointed with the fit when it arrived. i ordered the xs and it was still oversize to the point of being unflattering. i am tall 5'9 " about 130 pounds and have a fairly thin torso and look best in cloths that have some shape. if you like a loose fit this might be for you. the material is thicker and warm and comfortable. i would suggest ordering down a size.</s><s>so unflattering! really disappointed. made
----------------------------------------------------------------------------------------------------
 me look 6 month pregnant and i'm a petite size 2.</s><s>i love rompers and this one is really cute. i usually wear size 12 but should have got a 10, it runs big. it seems too long, and i'm 5'9 ". the prints cute but a little blah. i paid $ 158 which is too much, since i haven't worn it yet, i should have waited for it to go on sale.</s><s>... the print is so
------------------------------------------------------------------------------------------

In [ ]:
for i in tdc.main_ddict['validation']['input_ids'][:3]:
    print(tokenizer.decode(i))
    print('-'*100)

<s>this picture doesn't do the skirt justice. i paired it with a creme colored cashmere cowlneck sweater and a silver jeweled belt. it is really pretty and flattering on.</s><s>easy to wear! cute, comfy... will be a go to for summer.</s><s>nice sweater, just did not look good on me. sorry, going back.</s><s>this jacket was a little shorter than i had expected, but i still really enjoy the cut and fit of it
----------------------------------------------------------------------------------------------------
.</s><s>i wasn't planning on loving this dress when i tried it on. i loved the the color which is what prompted me to buy it. this dress fit perfectly. it hugs my body without feeling tight. the ruching is perfect. i didn't want to take it off! it's also very comfortable. i'm 5'1 ", 107 lbs and the xs petite fit perfectly. the dress hits me at the same length that is pictured. i think it would
--------------------------------------------------------------------------------------------

### Striding (For Concatenation of tokens)

If your sentences (or paragraphs) are larger than `max_length`, after concatenation, they will be broken apart; your long paragraph will be incompleted in terms of meaning. **Striding** is a way to somewhat preserve the sentence's meaning, by getting part of the sentence back. We will demonstrate it with an example, and you can compare it with the previous one (without striding) to see the differences

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataLMController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         content_transformations=[text_normalize,str.lower],
                         seed=42,
                         verbose=False,
                        )

In [ ]:
tdc.process_and_tokenize(tokenizer,line_by_line=False,max_length=100,stride=20)
# Stride is 20, meaning for the next entry, we go back 20 tokens

In [ ]:
for i in tdc.main_ddict['train']['input_ids'][:3]:
    print(tokenizer.decode(i))
    print('-'*100)

<s>i ordered this online and was disappointed with the fit when it arrived. i ordered the xs and it was still oversize to the point of being unflattering. i am tall 5'9 " about 130 pounds and have a fairly thin torso and look best in cloths that have some shape. if you like a loose fit this might be for you. the material is thicker and warm and comfortable. i would suggest ordering down a size.</s><s>so unflattering! really disappointed. made
----------------------------------------------------------------------------------------------------
 comfortable. i would suggest ordering down a size.</s><s>so unflattering! really disappointed. made me look 6 month pregnant and i'm a petite size 2.</s><s>i love rompers and this one is really cute. i usually wear size 12 but should have got a 10, it runs big. it seems too long, and i'm 5'9 ". the prints cute but a little blah. i paid $ 158 which is too much, since i haven't worn it
----------------------------------------------------------------

For the second entry, we can see it starts with the last 20 tokens of the previous entry: `comfortable. i would suggest ordering down a size.</s><s>so unflattering! really disappointed. made`)

In [ ]:
for i in tdc.main_ddict['validation']['input_ids'][:3]:
    print(tokenizer.decode(i))
    print('-'*100)

<s>this picture doesn't do the skirt justice. i paired it with a creme colored cashmere cowlneck sweater and a silver jeweled belt. it is really pretty and flattering on.</s><s>easy to wear! cute, comfy... will be a go to for summer.</s><s>nice sweater, just did not look good on me. sorry, going back.</s><s>this jacket was a little shorter than i had expected, but i still really enjoy the cut and fit of it
----------------------------------------------------------------------------------------------------
 was a little shorter than i had expected, but i still really enjoy the cut and fit of it.</s><s>i wasn't planning on loving this dress when i tried it on. i loved the the color which is what prompted me to buy it. this dress fit perfectly. it hugs my body without feeling tight. the ruching is perfect. i didn't want to take it off! it's also very comfortable. i'm 5'1 ", 107 lbs and the xs pet
---------------------------------------------------------------------------------------------

## Data Collator

In [ ]:
from underthesea import text_normalize
from transformers import AutoTokenizer

### For masked language model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

Let's define our text controller first

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataLMController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         content_transformations=[text_normalize,str.lower],
                         seed=42,
                         verbose=False
                        )

We will tokenize our corpus line-by-line

In [ ]:
tdc.process_and_tokenize(tokenizer,line_by_line=True,max_length=-1)

In [ ]:
tdc.set_data_collator(is_mlm=True,mlm_prob=0.15)

In [ ]:
tdc.data_collator

DataCollatorForLanguageModeling(tokenizer=RobertaTokenizerFast(name_or_path='roberta-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True), mlm=True, mlm_probability=0.15, pad_to_multiple_of=8, tf_experimental_compile=False, return_tensors='pt')

Before applying the collator...


In [ ]:
print([tdc.main_ddict['train'][i] for i in range(2)])

[{'input_ids': [0, 118, 2740, 42, 804, 8, 21, 5779, 19, 5, 2564, 77, 24, 2035, 479, 939, 2740, 5, 3023, 29, 8, 24, 21, 202, 81, 10799, 7, 5, 477, 9, 145, 29747, 24203, 479, 939, 524, 6764, 195, 128, 361, 22, 59, 8325, 2697, 8, 33, 10, 5342, 7174, 28762, 8, 356, 275, 11, 21543, 29, 14, 33, 103, 3989, 479, 114, 47, 101, 10, 7082, 2564, 42, 429, 28, 13, 47, 479, 5, 1468, 16, 33997, 8, 3279, 8, 3473, 479, 939, 74, 3608, 12926, 159, 10, 1836, 479, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'special_tokens_mask': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

We can see that the length of each token list is different from each other

In [ ]:
list(map(len,tdc.main_ddict['train']['input_ids'][:5]))

[91, 24, 79, 82, 121]

Let's apply the collator

In [ ]:
out = tdc.data_collator([tdc.main_ddict['train'][i] for i in range(5)]) # simulation with batch size 5

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
out.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

Now all token lists have the same length, which is 128: a multiple of 8 and larger than the longest list in the batch (which is 121)

In [ ]:
out['input_ids'].shape

torch.Size([5, 128])

In [ ]:
out['input_ids'][:2,:]

tensor([[    0,   118,  2740,    42,   804,     8, 50264,  5779, 50264,     5,
          2564,    77, 50264,  2035,   479, 50264,  2740, 50264,  3023,    29,
             8,    24,    21,   202,    81, 10799,     7,     5,   477, 50264,
         50264, 29747, 24203,   479, 50264,   524,  6764,   195,   128,   361,
            22,    59,  8325,  2697,     8,    33,    10,  5342,  7174, 28762,
             8,   356,   275,    11, 21543,    29,    14, 50264,   103, 50264,
           479, 50264,    47,   101,    10,  7082,  2564,    42,   429,    28,
         50264,    47,   479, 50264, 50264,    16, 33997, 50264,  3279,     8,
          3473,   479,   939,    74, 50264, 12926,   159,    10,  1836, 50264,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,  

The `labels` have also been constructed, which shows the "mask" tokens (non -100) in which the model has to predict. To increase the amount of masked tokens, increase the `mlm_prob`

In [ ]:
out['labels'][:2,:]

tensor([[-100, -100, -100, -100, -100, -100,   21, -100,   19, -100, -100, -100,
           24, -100, -100,  939, -100,    5, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100,    9,  145, -100, -100, -100,  939, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100,   33, -100, 3989,
         -100,  114, -100, -100, -100, -100, -100, -100, -100, -100,   13, -100,
         -100,    5, 1468, -100, -100,    8, -100, -100, -100, -100, -100, -100,
         3608, -100, -100, -100, -100,  479, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100],
        [-100, -100, -100, -100, -100, -100, 5779, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, 

If you apply padding in the tokenization step (by adjusting the `max_length` argument), no matter whether it's line-by-line tokenization or not, the data collator will skip the padding step

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataLMController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         content_transformations=[text_normalize,str.lower],
                         seed=42,
                         verbose=False
                        )

In [ ]:
tdc.process_and_tokenize(tokenizer,line_by_line=False,max_length=100)

In [ ]:
tdc.set_data_collator(is_mlm=True,mlm_prob=0.15)

In [ ]:
list(map(len,tdc.main_ddict['train']['input_ids'][:5]))

[100, 100, 100, 100, 100]

Let's apply the collator

In [ ]:
out = tdc.data_collator([tdc.main_ddict['train'][i] for i in range(5)]) # simulation with batch size 5

In [ ]:
out['input_ids'].shape

torch.Size([5, 100])

In [ ]:
out['input_ids'][:2,:]

tensor([[    0,   118,  2740,    42,   804,     8,    21,  5779,    19,     5,
          2564,    77, 22570,  2035,   479,   939,  2740,     5,  3023,    29,
             8,    24,    21,   202,    81, 50264,     7, 50264,   477,     9,
           145, 29747, 24203,   479,   939,   524,  6764,   195,   128,   361,
            22,    59,  8325,  2697, 50264,    33, 50264, 50264, 50264, 28762,
         31536,   356,   275,    11, 21543, 50264,    14,    33,   103,  3989,
           479, 50264,    47,   101,    10,  7082,  2564,    42,   429,    28,
            13,    47,   479,     5,  1468, 34456, 50264,     8,  3279,     8,
          3473,   479,   939,    74,  3608, 12926,   159,    10,  1836,   479,
             2,     0,  2527, 29747, 24203, 27785,   269,  5779,   479, 50264],
        [  162,   356,   231,   353,  5283,     8,   939, 50264,    10,  4716,
          1459,  1836,   132,   479,     2,     0,   118,   657, 50264,  7474,
           268,     8,    42,    65,    16,   269, 

In [ ]:
out['labels'][:2,:]

tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          2564,  -100,    24,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100, 10799,  -100,     5,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  6764,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,     8,  -100,    10,  5342,  7174,  -100,
             8,  -100,  -100,  -100,  -100,    29,  -100,  -100,  -100,  -100,
          -100,   114,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,    16, 33997,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,   156],
        [ -100,  -100,  -100,  -100,  -100,  -100,  -100,   437,    10,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,   910,  -100,
          -100,  -100,  -100,  -100,  -100,  -100, 

Since we are using the concatenation-of-tokenization technique, one smart thing that the HuggingFace's `DataCollatorForLanguageModeling` (which is the data collator we use) does is to allow maskings at every position, at opposed to to the previous cases (with line-by-line tokenization), there's no masking near the end tokens of each list, because those end tokens are padding tokens

### For causal language model

In [ ]:
from transformers import AutoTokenizer
from tokenizers import processors

Let's define our GPT2 tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')

In [ ]:
tokenizer

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)

GPT2 does not use start/end-of-sentence token:

In [ ]:
print(tokenizer.convert_ids_to_tokens(tokenizer("this is a text. That is a second text.But there's a third one")['input_ids']))

['this', 'Ġis', 'Ġa', 'Ġtext', '.', 'ĠThat', 'Ġis', 'Ġa', 'Ġsecond', 'Ġtext', '.', 'But', 'Ġthere', "'s", 'Ġa', 'Ġthird', 'Ġone']


If you want to perform concatenation-of-token, and you want your causal LM to differentiate between sentences, you can add a special token to separate sentences, as follow:

In [ ]:
tokenizer._tokenizer.post_processor = processors.TemplateProcessing(
    single="$A " + tokenizer.eos_token,
    special_tokens=[(tokenizer.eos_token, tokenizer.eos_token_id)],
)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
print(tokenizer.convert_ids_to_tokens(tokenizer("this is a text. That is a second text.But there's a third one")['input_ids']))

['this', 'Ġis', 'Ġa', 'Ġtext', '.', 'ĠThat', 'Ġis', 'Ġa', 'Ġsecond', 'Ġtext', '.', 'But', 'Ġthere', "'s", 'Ġa', 'Ġthird', 'Ġone', '<|endoftext|>']


With this modified tokenizer, let's perform concatenation-of-tokenization using GPT2

In [ ]:
dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataLMController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         content_transformations=[text_normalize,str.lower],
                         seed=42,
                         verbose=False
                        )

In [ ]:
tdc.process_and_tokenize(tokenizer,line_by_line=False,max_length=100)

Since it's casual language modeling, let's turn off `is_mlm`

In [ ]:
tdc.set_data_collator(is_mlm=False)

In [ ]:
list(map(len,tdc.main_ddict['train']['input_ids'][:5]))

[100, 100, 100, 100, 100]

Let's apply the collator

In [ ]:
out = tdc.data_collator([tdc.main_ddict['train'][i] for i in range(5)]) # simulation with batch size 5

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
out['input_ids'].shape

torch.Size([5, 100])

In [ ]:
out['input_ids'][:2,:]

tensor([[   72,  6149,   428,  2691,   290,   373, 11679,   351,   262,  4197,
           618,   340,  5284,   764,  1312,  6149,   262,  2124,    82,   290,
           340,   373,   991,   625,  7857,   284,   262,   966,   286,   852,
         42880, 16475,   764,  1312,   716,  7331,   642,   705,   860,   366,
           546, 11323,  8059,   290,   423,   257,  6547,  7888, 28668,   290,
           804,  1266,   287, 16270,    82,   326,   423,   617,  5485,   764,
           611,   345,   588,   257,  9155,  4197,   428,  1244,   307,   329,
           345,   764,   262,  2587,   318, 29175,   290,  5814,   290,  6792,
           764,  1312,   561,  1950, 16216,   866,   257,  2546,   764, 50256,
           568, 42880, 16475,  5145,  1107, 11679,   764,   925,   502,   804],
        [  718,  1227, 10423,   290,  1312,  1101,   257,  4273,   578,  2546,
           362,   764, 50256,    72,  1842,   374,  3361,   364,   290,   428,
           530,   318,  1107, 13779,   764,  1312, 

In [ ]:
out['labels'][:2,:]

tensor([[   72,  6149,   428,  2691,   290,   373, 11679,   351,   262,  4197,
           618,   340,  5284,   764,  1312,  6149,   262,  2124,    82,   290,
           340,   373,   991,   625,  7857,   284,   262,   966,   286,   852,
         42880, 16475,   764,  1312,   716,  7331,   642,   705,   860,   366,
           546, 11323,  8059,   290,   423,   257,  6547,  7888, 28668,   290,
           804,  1266,   287, 16270,    82,   326,   423,   617,  5485,   764,
           611,   345,   588,   257,  9155,  4197,   428,  1244,   307,   329,
           345,   764,   262,  2587,   318, 29175,   290,  5814,   290,  6792,
           764,  1312,   561,  1950, 16216,   866,   257,  2546,   764,  -100,
           568, 42880, 16475,  5145,  1107, 11679,   764,   925,   502,   804],
        [  718,  1227, 10423,   290,  1312,  1101,   257,  4273,   578,  2546,
           362,   764,  -100,    72,  1842,   374,  3361,   364,   290,   428,
           530,   318,  1107, 13779,   764,  1312, 

For CLM, the `labels` are essentially the same as `input_ids`. From HuggingFace documentation:
```
`DataCollatorForLanguageModeling` will take care of creating the language model labels — in causal language modeling the inputs serve as labels too (just shifted by one element), and this data collator creates them on the fly during training.
```

## Save and Load TextDataController

In [ ]:
show_doc(TextDataLMController.save_as_pickles)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main_lm.py#L62){target="_blank" style="float:right; font-size:smaller"}

### TextDataLMController.save_as_pickles

>      TextDataLMController.save_as_pickles (fname, parent='pickle_files')

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| fname |  |  | Name of the pickle file |
| parent | str | pickle_files | Parent folder |

In [ ]:
show_doc(TextDataLMController.from_pickle)

---

[source](https://github.com/anhquan0412/that-nlp-library/blob/main/that_nlp_library/text_main.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### TextDataController.from_pickle

>      TextDataController.from_pickle (fname, parent='pickle_files')

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| fname |  |  | Name of the pickle file |
| parent | str | pickle_files | Parent folder |

TextDataLMController object can be saved and loaded with ease. This is especially useful after text processing and/or tokenization have been done

In [ ]:
from datasets import disable_caching

In [ ]:
disable_caching()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

dset = load_dataset('sample_data',data_files=['Womens_Clothing_Reviews.csv'],split='train')
tdc = TextDataLMController(dset,
                         main_text='Review Text',
                         filter_dict={'Review Text': lambda x: x is not None},
                         content_transformations=[text_normalize,str.lower],
                         seed=42,
                         verbose=False
                        )

In [ ]:
tdc.process_and_tokenize(tokenizer,line_by_line=True,max_length=-1)

tdc.set_data_collator(is_mlm=True,mlm_prob=0.15)

In [ ]:
tdc.save_as_pickles('my_lm_tdc')

Load back our object

In [ ]:
tdc2 = TextDataLMController.from_pickle('my_lm_tdc')

You can still access all its attributes, data, preprocessings, transformations ...

In [ ]:
tdc2.main_ddict

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 18111
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'special_tokens_mask'],
        num_rows: 4529
    })
})

In [ ]:
tdc2.filter_dict,tdc2.content_tfms

({'Review Text': <function __main__.<lambda>(x)>},
 [<function underthesea.pipeline.text_normalize.text_normalize(text, tokenizer='underthesea')>,
  <method 'lower' of 'str' objects>])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()